In [2]:
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient
from getpass import getpass

#Connect to the elastic cloud server
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY, # your username and password for connecting to elastic, found under Deplouments - Security
)

/var/folders/k3/sm4kh0y91fs5_fft4375wwh00000gn/T/ipykernel_81533/2551640357.py:2: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


In [3]:
import os

index_name = "newsgroup"
model_id = "elastic/distilbert-base-cased-finetuned-conll03-english"

os.environ['ELASTIC_CLOUD_ID'] = ELASTIC_CLOUD_ID
os.environ['ELASTIC_API_KEY'] = ELASTIC_API_KEY
os.environ['MODEL_ID'] = model_id

In [19]:
!docker run -it --rm elastic/eland \
    eland_import_hub_model \
      --cloud-id $ELASTIC_CLOUD_ID \
      --es-api-key $ELASTIC_API_KEY \
      --hub-model-id $MODEL_ID \
      --task-type text_classification \
      --start 

2024-03-20 10:37:46,734 INFO : Establishing connection to Elasticsearch
2024-03-20 10:37:46,819 INFO : Connected to cluster named 'fdcc4e10e5a34385884a3eda9350099a' (version: 8.9.0)
2024-03-20 10:37:46,821 INFO : Loading HuggingFace transformer tokenizer and model 'elastic/distilbert-base-cased-finetuned-conll03-english'
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at elastic/distilbert-base-cased-finetuned-conll03-english and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.9/dist-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this func

In [17]:
! eland_import_hub_model --es-api-key $ELASTIC_API_KEY \
  --url $ELASTIC_CLOUD_ID \
  --hub-model-id $MODEL_ID \
  --task-type ner \
  --start

2024-03-20 11:29:19,361 ERROR : Failed to run because module 'torch' is not available.

This script requires PyTorch extras to run. You can install these by running:

    /Users/iulia/python/cheat sheet/elasticsearch-python/.venv/bin/python3.11 -m pip install 'eland[pytorch]'



In [1]:
models = MlClient.get_trained_models(es)
for model in models["trained_model_configs"]:
    print(model["model_id"])

NameError: name 'MlClient' is not defined